In [22]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
# Read the data

df = pd.read_csv('data/CommViolPredUnnormalizedData.txt', encoding='latin-1',header=None)
df.columns
#the column names are the second word in each row of data/crime_headings.txt
with open('data/crime_headings.txt') as f:
    headings = f.readlines()
col_names = []
types = []
for heading in headings:
    if len(heading.split()) <= 1:
        continue
    col_names.append(heading.split()[1])
    if heading.split()[2] == 'numeric':
        types.append(float)
    else:
        types.append(str)

df.columns = col_names

#drop drop rows with "?" values
df = df.replace('?', np.nan)

df = df.astype(dict(zip(col_names, types)))

#communityname, countyCode, communityCode, fold are not predictive so drop them
df = df.drop(['communityname', 'countyCode', 'communityCode', 'fold'], axis=1)
df.head()







,State,pop,perHoush,pctBlack,pctWhite,pctAsian,pctHisp,pct12-21,pct12-29,pct16-24,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,violentPerPop,nonViolPerPop
0,NJ,11980.0,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,PA,23123.0,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,OR,29344.0,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,NY,16656.0,2.40,1.70,97.35,0.50,0.70,12.55,25.20,12.19,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,MN,11245.0,2.76,0.53,89.16,1.17,0.52,24.46,40.53,28.69,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


In [86]:
#useful functions
def drop_rows_missing_target(df):
    return df.dropna(subset=['nonViolPerPop'])


def fill_with_mean(df):
    for column in df.columns:
        if df[column].dtype == float:
            df[column] = df[column].fillna(df[column].mean())
    return df

def fill_with_median(df):
    for column in df.columns:
        if df[column].dtype == float:
            df[column] = df[column].fillna(df[column].median())
    return df

def convert_categorical_to_numeric(df):
    for column in df.columns:
        if df[column].dtype != float:
            df[column] = df[column].astype('category')
            df[column] = df[column].cat.codes
    return df

def normalize(df):
    for column in df.columns:
        if df[column].dtype == float:
            df[column] = (df[column] - df[column].mean()) / df[column].std()
    return df

def remove_random_features(df, n):
    dropped_cols = np.random.choice(df.columns[:-1], n, replace=False)
    return df.drop(dropped_cols, axis=1), dropped_cols

def feature_selection(df):
    #drop the last 18 columns
    X = df.drop(df.columns[-1:], axis=1)
    y = df['nonViolPerPop']
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X, y)
    importance = rf.feature_importances_
    # print(np.sort(importance))
    indices = np.argsort(importance)[::-1]
    kept_cols = []
    for f in range(X.shape[1]):
        if importance[indices[f]] > 0.1:
            kept_cols.append(X.columns[indices[f]])
    #put nonViolPerPop back in
    kept_cols.append('nonViolPerPop')
    return df[kept_cols], kept_cols


# Black Box Approach

In [87]:
black_box_df = fill_with_mean(normalize(drop_rows_missing_target(convert_categorical_to_numeric(df))))

black_box_df, kept_cols = feature_selection(black_box_df)

train, test = train_test_split(black_box_df, test_size=0.2)

X_train = train.drop(train.columns[-1], axis=1)
y_train = train['nonViolPerPop']
X_test = test.drop(test.columns[-1], axis=1)
y_test = test['nonViolPerPop']

# Create the model with 100 trees
model = RandomForestRegressor(n_estimators=100,
                                bootstrap = True,
                                max_features = 'sqrt')
# Fit on training data

model.fit(X_train, y_train)

# Actual class predictions
rf_predictions = model.predict(X_test)

#calculate mae
mae = np.mean(abs(rf_predictions - y_test))
print('Mean Absolute Error:', mae)
# print('Kept columns:', kept_cols)
print()


Mean Absolute Error: 0.261596479984286



In [85]:
# https://www.geeksforgeeks.org/random-forest-regression-in-python/

for i in range(7):
    if i != 0:
        black_box_df, dropped_cols = remove_random_features(df, 10)
    else: 
        black_box_df = df
        dropped_cols = None

    black_box_df = fill_with_mean(normalize(drop_rows_missing_target(convert_categorical_to_numeric(black_box_df))))

    train, test = train_test_split(black_box_df, test_size=0.2)
    X_train = train.drop(train.columns[-1], axis=1)
    y_train = train['nonViolPerPop']
    X_test = test.drop(test.columns[-1], axis=1)
    y_test = test['nonViolPerPop']

    # Create the model with 100 trees
    model = RandomForestRegressor(n_estimators=100,
                                    bootstrap = True,
                                    max_features = 'sqrt')
    # Fit on training data
    model.fit(X_train, y_train)

    # Actual class predictions
    rf_predictions = model.predict(X_test)

    #calculate mae
    mae = np.mean(abs(rf_predictions - y_test))
    print('Mean Absolute Error:', mae)
    print('Dropped Columns:', dropped_cols)
    print()
    




Mean Absolute Error: 0.2431847131586867
Dropped Columns: None

Mean Absolute Error: 0.21280952853114957
Dropped Columns: ['kidsBornNevrMarr' 'burglaries' 'pctUrban' 'pctSameState-5' 'pctKids2Par'
 'landArea' 'medNumBedrm' 'policePerPop' 'policCallPerPop'
 'pctVacantBoarded']

Mean Absolute Error: 0.2111100870324186
Dropped Columns: ['pctWsocsec' 'rentUpperQ' 'medYrHousBuilt' 'murders' 'pctNotHSgrad' 'pop'
 'policeField' 'blackPerCap' 'pctEmployProfServ' 'pct2Par']

Mean Absolute Error: 0.24958373034487166
Dropped Columns: ['pctSameHouse-5' 'perHoush' 'pctMaleDivorc' 'pctLowEdu'
 'persPerOccupHous' 'assaults' 'kidsBornNevrMarr' 'rapes' 'pctImmig-3'
 'pctPoverty']

Mean Absolute Error: 0.2312434574229637
Dropped Columns: ['robberies' 'pctFgnImmig-8' 'pctPolicPatrol' 'pct12-21'
 'pctVacantBoarded' 'pct65up' 'larcenies' 'burglaries' 'ownHousLowQ'
 'pctBlack']

Mean Absolute Error: 0.20337759530272734
Dropped Columns: ['pct65up' 'perCapInc' 'rentLowQ' 'numDiffDrugsSeiz' 'ownHousUperQ'
 'pol

# Explainable Approach

In [ ]:
# find best features using best subset selection
#Elastic net regression?
#variable selection
#lasso


